In [5]:
from sage.interfaces.gap import  get_gap_memory_pool_size, set_gap_memory_pool_size
set_gap_memory_pool_size(24364842180)

'''Graded characters in the regular block O_0.
Note: Notation is dominant: L(e) is finite dimensional, Delta(e) is projective, Delta(w0) = L(w0), ...'''


W = WeylGroup("D5", prefix="s")
[s1,s2,s3,s4,s5] = W.simple_reflections()

##################################################################################

n = rank(W)
w0 = W.long_element()
e = W(1)


####### Kazhdan-Lusztig polynomials ##########################

# A faster implementation of KL-polynomials (using the optional package Coxeter 3) is given by this
# Fokko Ducloux’s Coxeter3 C++ library.

# Had to install it: I just typed "sage -i coxeter3" in the terminal.

# It seems that one can direcly coerce from WeylGroup to CoxeterGroup and vice versa.
# I will therefore use CoxeterGroup to calculate KL-polynomials, but for all other Bruhat business I will use WeylGroup.

R.<q> = LaurentPolynomialRing(QQ)

KL = KazhdanLusztigPolynomial(W,q)  # KL-polynomials implemented in standard Sage way
# http://doc.sagemath.org/html/en/reference/combinat/sage/combinat/kazhdan_lusztig.html


CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")

def KLP(x,y):
    '''Returns the KL-polynomial, implemented in "Coxeter3" package by Fokko du Cloux.
    http://math.univ-lyon1.fr/~ducloux/coxeter/coxeter3/english/coxeter3_e.html'''
    
    if x not in W:
        x = convert_from_123(x)
    if y not in W:
        y = convert_from_123(y)
    
    return CoxeterPackage.kazhdan_lusztig_polynomial(CoxeterPackage(x), CoxeterPackage(y))
    # If "coxeter3" is not installed, remove the line 'CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")'
    # and in this function return KL.P(x,y)
    #return KL.P(x,y)

#Point:
#    - standard Sage way: KL.P(x,y)
#    - faster way: KLP(x,y) 


R.<v> = LaurentPolynomialRing(QQ)
def KLPv(x,y):
    '''v-normalization: degree in the polynomial agrees with the graded degree in the Verma module.'''
    return KLP(x, y)(q=1/v^2) * v^(y.length()-x.length())


def mu(w,x):
    '''Returns the KL mu-function with arguments w,x.
    By Humphrey's BGG book p. 175 and p. 169, for w<x we have:
    mu(x,w) = mu(w0*w,w0*x),
    mu(w,x) = dim Exit^1 (L_w,L_x) = dim Exit^1(L_x,L_w) = dim Exit^1(Delta_x,L_w).'''

    if w not in W:
        w = convert_from_123(w)
    if x not in W:
        x = convert_from_123(x)
        
    if w.bruhat_le(x):
        poly_dict = KLP(w,x).dict()       
        j = (x.length()-w.length()-1)/2 
        if j not in poly_dict.keys():
            return 0
        return poly_dict[j]

    return 0


def dimExt(i,y,w):  
    '''Returns dimExt^i(Delta(y),L(w)).'''
    
    y = y*w0                               
    w = w*w0   
    poly_dict = KLP(y,w).dict()       
    j = (w.length()-y.length()-i)/2 
    if j not in poly_dict.keys():
        return 0
    return poly_dict[j]


def mult_Delta_L(y,w):
    '''Returns the Jordan-Holder multiplicity [Delta(y),L(w)] in O_0.
    This is quicker than the analogous operation on GradedChar classes because
    it does not calculate the full Delta.'''
      
    return KLP(y,w)(1)               


def mult_graded_Delta_L(w,k,x):
    '''Returns the graded Jordan-Holder multiplicity [Delta(w) shifted by k,L(x)].
    This is quicker than the analogous operation on GradedChar classes because
    it does not calculate the full Delta.''' 

    l_wx = x.length() - w.length()
    
    i = (l_wx-k)
    if i<0 or i%2!=0:
        return 0
    i = i/2
    
    KL_wx = KLP(w, x).dict()
    if i not in KL_wx.keys():
        return 0
    
    return KL_wx[i]


def earliest_occurence(x):
    '''Returns the smallest non-negative k such that L(x) occurs in Delta(e) at level k.
    To be used later in Lusztig's a function.'''
      
    for k in range(x.length()+1):
        
        if mult_graded_Delta_L(e,k,x) > 0:
            return k
    

##### Parabolic subgroups


def W_(Sigma):
    '''Parabolic subgroup W_Sigma as a list.'''
    
    return W.bruhat_interval(e,w0_(Sigma))


def minimal_rep(x,Sigma):
    '''The minimal representative of the class x*W_Sigma.'''
    
    return x.coset_representative(Sigma, side='right')


def w0_(Sigma):
    '''Returns the longest element of the parabolic subgroup defined by Sigma.
    Format of sigma: e.g. Sigma = [1,2,4] returns s4*s1*s2*s1.'''
    
    return (w0.coset_representative(Sigma, side='right')).inverse() * w0


def Maximal_elements_of_parabolics():
    '''Returns a list of the longest elements of all parabolic subgroups.'''
    
    return [w0_(Sigma) for Sigma in Subsets(range(1,rank(W)+1))]



######### Some operations on graded characters #########


def remove_keys_with_value(dict,value):
    '''Removes all items from dict with given value. Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().remove_zeros.'''
    
    remove = []
    for x in dict:
        if dict[x] == value:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

        
def remove_keys_with_value_smaller_than(dict,bound):
    '''Removes all items from dict with value smaller than or equal to "bound".
    Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().only_positive.'''
    
    remove = []
    for x in dict:
        if dict[x] <= bound:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

    
def format_star(string):
    '''Windows cannot have * in filenames, so need to use this function when saving files.
    This will replace "*" with "_".'''
    
    new_string = string.replace("*", "_")
    return new_string


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return str(w).replace("s","").replace("*","")


def convert_from_123(string):
    '''Converts one element from W in the "123" string notation to the usual "s1*s2*s3" notation.'''
    
    if type(string)== Integer:
        string = str(string)
        
    if string == "e":
        return W(1)
    
    string = "*".join([char for char in string])
    
    for i in range(1,n+1):
        string = string.replace(str(i),"s%s"%i)
    
    return eval(string)


def convert_to_123_long(string):
    '''E.g. For input "4*s1*s2*s3", the output is "4*123".
    Not used for now.'''
    
    string_e = ""  # This should be a copy of string, but for "e" instead of each "1" that represent the trivial composition factor.
    
    for i in range(len(string)):
        
        condition = 0
        
        if string[i] == "1" and i>0:
            
            if string[i-1] not in ["s","-","0","1","2","3","4","5","6","7","8","9"]:
                
                if i+1==len(string):
                    condition = 1
                
                else:
                    if string[i+1] not in ["*",":","0","1","2","3","4","5","6","7","8","9"]:
                        condition = 1
    
        if condition == 0:
            string_e += string[i]
        
        else:
            string_e += "e"
    
        
    string_no_ast = re.sub(r'(?<=(?<=s).)\*',"",string_e) # Remove "*" if there is "s" two places before.
    string_no_ast_s = string_no_ast.replace("s","")
    
    return string_no_ast_s


class GradedChar:
    '''A class representing a graded character of module in the graded version of category O.
    self.graded is a dictionary where keys are indices of the graded components.
    Each graded component is a dictionary with keys being composition factors in that graded piece, and values are multiplicities.'''
    
    def __init__(self):
        self.component = {}
        self.name = ""
           
    def __str__(self):
        '''Each line becomes graded piece.'''
        
        if self == char_0():   # For zero character, print "0" instead of nothing.
            return "0"
        
        str_glob = ""
        if self.name != "":
            str_glob += self.name +":\n\n"
        
        for i in sorted(self.component.keys()):
            lis_i = []
            
            for w in (self.component[i]).keys():
                
                m = self.component[i][w]    # Multiplicity of w in i-th graded piece of self.
                m_string = ""
                if m != 1:
                    m_string = "%d*"%m      # Needed to print nicelly formated graded pieces.
                
                lis_i += ['%s'%m_string + str(w)]
            
            str_glob += "%d: "%i + ", ".join(lis_i) + "\n"
        
        return str_glob

    def __eq__(self, other): 
        if not isinstance(other, GradedChar):
            # Don't attempt to compare against unrelated types.
            return NotImplemented
        
        self.remove_zeros()      # Remove all the redundancies before comparing.
        other.remove_zeros()
        
        return self.component == other.component

    def __add__(self,char2):    
        char_sum = GradedChar()

        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_sum.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_sum.component[k][w] = self_k.get(w,0) + char2_k.get(w,0)

        return char_sum
    
    def __mul__(self,other):
        prod = GradedChar()

        for k in self.component:
            prod.component[k] = {}
        
            for w in self.component[k]:
                prod.component[k][w] = other * self.component[k][w]

        return prod
    
    def __rmul__(self,other):
        return self*other
    
    def __sub__(self,other):
        return self+((-1)*other)
    
    def __neg__(self):
        return (-1)*self
    
    
    def rename(self,new_name):
        '''Changes the name of self, and returns new_name.'''
        
        self.name = new_name
        return self.name
    
    
    def remove_zeros(self):
        '''Removes all composition factors with coefficients zero.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self
        
    
    def only_positive(self):
        '''Removes all composition factors with non-positive coefficients.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value_smaller_than(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self    
    
    
    def add_factor(self,i,w):
        '''Adds a composition factor to self in i-th graded piece.'''
        
        if i not in self.component:
            self.component[i] = {}
        
        if w not in self.component[i]:
            self.component[i][w] = 0
        
        self.component[i][w] += 1
        
        self.name = ""    # The character "self" has changed, so the old name is not valid anymore.
        
        return self

    
    def is_simple(self):
        '''Checks whether self is a simple module.'''

        self.remove_zeros()

        if len(components(self)) != 1:
            return False

        only_component = self.component[ list(self.component.keys())[0] ]

        if len(only_component.keys()) != 1:
            return False

        only_factor = list(only_component.keys())[0]

        if only_component[ only_factor ]  != 1:
            return False

        return True
   
    
    def is_true_character(self):
        '''Returns True iff all the coefficients of self are non-negative.
        Assumes that all the coeffictients are integers.'''
        
        for i in self.component:
            for w in self.component[i]:
                if not ( self.component[i][w] >= 0 ):
                    return False
        return True
  

    def min_character(self,char2):
        '''Returns the character whose components have multiplicities equal to minimum
        of the multiplicities of the corresponding components in self and char2.'''
        
        char_min = GradedChar()
       
        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_min.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_min.component[k][w] = min(self_k.get(w,0),char2_k.get(w,0))

        return char_min    
    

    
######### Some operations on graded characters #########


def shift(char,i):
    '''Returns the shift by i of char.'''
    
    char_new = GradedChar()
    
    for key in char.component.keys():
        char_new.component[key-i] = char.component[key]
    
    return char_new


def dual(char):
    '''Returns the graded dual of char.
    Input: GradedChar.'''
    
    char_dual = GradedChar()
    
    for key in char.component.keys():
       char_dual.component[-key] = char.component[key]
    
    return char_dual


def components(char):
    '''Returns the list of indexes of non-zero graded pieces of char.'''
    
    char.remove_zeros()
    return sorted(char.component.keys())


def truncate(char,lis):
    '''Cuts off from char all graded pieces from lis.'''
    
    for k in lis:
        char.component.pop(k, None)
    
    return char
    
    
def mult(char,i,w):
    '''Returns the multiplicity of w in i-th graded piece of char.'''
    
    if i not in char.component:
        return 0
    
    return char.component[i].get(w,0)


def total_mult(char,w):
    '''Returns the total multiplicity of w in char.'''
    
    return sum(char.component[i].get(w,0) for i in char.component)


def dict_mult(char,w):
    '''Returns dictionary with items (k:m), where m is the multiplicity of w in k-th graded piece of char.'''
    
    dic = {}
    for k in char.component:
        if char.component[k].get(w,0) != 0:
            dic[k] = char.component[k][w]
            
    return dic


def clean(char):
    '''Returns cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    char_dummy = GradedChar()
    
    char_dummy.name = (char.name).replace("(1)","(e)").replace("s","").replace("*","")
    
    for k in char.component:
        char_dummy.component[k] = {}
        
        for w in char.component[k]:
            char_dummy.component[k][convert_to_123(w)] = char.component[k][w]
    
    return(str(char_dummy))


def print_clean(char):
    '''Prints cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    print(clean(char))
    

def combinatorial_cokernel(char1, char2):
    '''Returns the lower bound on the cokernel of a map from X to Y.'''
    
    virtual = char2-char1
    virtual.only_positive()
    
    return virtual


def combinatorial_kernel(char1, char2):
    '''Returns the lower bound on the kernel of a map from X to Y.'''
    
    virtual = char1-char2
    virtual.only_positive()
    
    return virtual



def ungrade(X):
    '''Returns the ungraded version of graded character X: all composition factors are moved to degree 0.'''
    
    unX = char_0()
    
    for i in components(X):
        unX += shift(X,i)
    
    truncate(unX,[i for i in components(unX) if i != 0])
    
    return unX



######### Some graded characters #########


def char_0():
    '''Returns the zero graded character.'''
    
    zero = GradedChar()
    zero.rename('0')
    return zero


def char_L(w):
    '''Returns the graded character of the simple L(w).'''
    
    L = GradedChar()
    L.add_factor(0,w)
    L.rename('L(%s)'%w)
    return L

        
def char_Delta(w):
    '''Returns the graded character of the Verma Delta(w).
    Reference: Humphrey's BGG book, Thm p. 175.'''
        
    Delta = GradedChar()
    
    const_name = 'Delta(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        Delta.component = const_dict
        Delta.rename(const_name)
        return Delta
    
    for x in W.bruhat_interval(w,w0):

        l_wx = x.length() - w.length()
        KL_wx = KLP(w, x).dict()
        for i in KL_wx.keys():
            for j in range(KL_wx[i]):
                Delta.add_factor(l_wx-2*i, x)

    Delta.rename(const_name)    
    return Delta
     

def char_Nabla(w):
    '''Returns the graded character of the dual Verma Nabla(w).'''
    
    Nabla = dual(char_Delta(w))
    Nabla.rename('Nabla(%s)'%w)
    return Nabla
    

def char_P(w):
    '''Returns the graded character of the indecomposable projective P(w).
    Uses graded BGG reciprocity.'''
    
    P = GradedChar()

    const_name = 'P(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        P.component = const_dict
        P.rename(const_name)
        return P
    
    for x in W.bruhat_interval(e,w):
        
        Delta = char_Delta(x)
        
        dic = dict_mult(Delta,w)
        
        for k in dic:
            P += shift(Delta,-k)*dic[k]

    P.rename('P(%s)'%w)
    return P
        
    
def char_I(w):
    '''Returns the graded character of the indecomposable injective I(w).'''
    
    I = dual(char_P(w))     
    I.rename('I(%s)'%w)
    
    return I
        

def char_T(w):
    '''Returns the graded character of the indecomposable tilting module T(w).
    Uses (graded) Soergel reciprocity.
    See Humphrey's BGG book, p. 233.
    This can also be proved by applying translation to graded BGG reciprocity.'''
    
    T = GradedChar()
    
    const_name = 'T(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        T.component = const_dict
        T.rename(const_name)
        return T
    
    for x in W.bruhat_interval(w,w0):
        dict = dict_mult(char_Delta(w0*x),w0*w)
        
        for k in dict:
            T += shift(char_Delta(x),k)*dict[k]
    
    T.rename('T(%s)'%w)
    return T    
    

def graded_char_BGG(w):
    '''Returns the graded character of the BGG complex of L(w).'''
    
    sum = char_0()
    l_w = w.length()
    
    for x in W.bruhat_interval(w,w0):
        l_x = x.length()
        
        sum += ((-1)^(l_x-l_w))*shift(char_Delta(x),-l_x+l_w)
        
    return sum



################ Projective functors ################


def theta_simple_simple(s,y):        
    '''Returns the graded character of theta_s(L_y), where s is a simple reflection,
    and y an element from W (not of class GradedChar).
    Reference: Coulembier-Mazorchuk-Zhang, Proposition 16,
    plus the fact that y and z are comparable if z appears.'''
        
    result = GradedChar()
    
    if y.bruhat_le(y*s):
        return result
    
    result.add_factor(-1,y)
    result.add_factor(1,y)
    result.add_factor(0,y*s)
    
    for z in W.bruhat_interval(y,w0):
        
        if z.bruhat_le(z*s):
            m = mu(y,z)
            if m != 0:
                result += m*char_L(z)
    
    return result


def theta_simple(s,M):
    '''Returns the graded character of theta_s(M), where s is a simple reflection,
    and M is from class GradedChar.
    Uses the function "theta_simple_simple" and the fact that theta is exact functor.'''
        
    result = GradedChar()
    
    for k in M.component:
        for w in M.component[k]:
            result += shift(M.component[k][w] * theta_simple_simple(s,w),  -k)
    
    return result


def theta(ws, char):          
    '''Returns the graded character of theta_ws(char).
    Uses recursion from [Mazorchuk: Some homolgical properties ... I, the proof of Theorem 11.a)].'''
    
    result = GradedChar()
    
    if char.name != "":
        const_name = 'theta_%s(%s)'%(ws,char.name)
        const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.

        if const_dict != None:
            
            result.component = const_dict
            result.rename(const_name)

            return result
    
    factors = ws.reduced_word()
    
    if len(factors) == 0:    # theta_e = Identity
        return char
    
    if char == char_L(w0):   # theta_x(L(w0)) = T(w0*x)
        result = char_T(w0*ws)
        result.rename(const_name)
        return result
    
    s = W.simple_reflections()[factors[len(factors)-1]]
    
    if len(factors) == 1:    # basis of the recursion
        
        result = theta_simple(s,char)
        
        if char.name != "":
            result.rename(const_name)
        return result
    
    w = ws * s
        
    subtract = [ y for y in W.bruhat_interval(e,w) if (y*s).bruhat_le(y) and mu(y,w) !=0  ]
    
    result = theta_simple(s, theta(w,char))
    
    for y in subtract:
        result = result - mu(y,w)*theta(y,char)
    
    if char.name != "":
        result.rename(result.rename(const_name))
    return result


def M(x,y):
    '''Different notation for theta_x(L(y)).'''
    
    return theta(x,char_L(y))


def th(a,b):
    '''Same as theta, but uses 123-notation in number format, not as strings.
    Also prints theta along the way.'''
    
    x = convert_from_123(str(a))
    y = convert_from_123(str(b))
    
    X = theta(x,char_L(y))
    print_clean(X)

    return  X



######### Saving and reading characters to/from external files #########


import os
import datetime
import re    # Regular expressions!

    
def save_(char):
    '''Saves char to a file in the folder.
    In fact, it saves 
    1. "char.component" dictionary to "./Graded_characters_in_O_0_data/Modules/'type'/", under the name "dict_'name'.txt",
    2. The output of print(char) string to "./Graded_characters_in_O_0_data/For_human/'type'/", under the name "'name'.txt".
    3. The output of print_clean(char) string to "./Graded_characters_in_O_0_data/For_human123/'type'/".
    Asterixes (*) are omitted from filenames.'''

    if char.name == "":
        return "Error: char has no name."        # For now I do not want to save characters without a name.
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    
    folder1 = 'Graded_characters_in_O_0_data'
    For_computer = 'Modules'    
    path_computer = folder1 +'/' + For_computer     
    path_computer_CT = path_computer + '/' + CT
    
    for path in [folder1, path_computer, path_computer_CT]: # Check if necessary folders exist, and create them if not.

        if not os.path.isdir(path):  # Is there a folder already?
            os.mkdir(path)        # If not, create one.
    
    name_without_asterix = format_star(char.name)     # remove asterixes from name

    file_computer = open(path_computer_CT + "/dict_%s.txt"%name_without_asterix, "w+")        
    file_computer.write(str(char.component))     # Saves as a dictionary - usefull if I want to load back.    
    file_computer.close()  
    

def is_saved(name, option1):    # option1 should be "read_it" or "only_bool".
    '''Checks if the character with "name" has already been saved.
    If option1=="read_it": If no file, returns None. If yes, returns the dictionary.
    If option1=="only_bool": If no file, returns False. If yes, returns True.'''
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])    # The names must be the same as in "save_(char)" function.
    
    folder1 = 'Graded_characters_in_O_0_data'
    For_computer = 'Modules'
    
    name = format_star(name)    # remove asterixes from name
    
    path_computer_CT = folder1 +'/' + For_computer + '/' + CT + '/dict_' + name + '.txt'
    
    if not os.path.isfile(path_computer_CT):   # If there is no file.
        
        if option1 == "read_it":
            return None
        
        if option1 == "only_bool":
            return False
    
    # Here we assume that the file exists.
    
    if option1 == "only_bool":
        return True
    
    # Here option1 == "only_bool" and the file exists, so we want to read it and return it.
    
    file = open(path_computer_CT, "r")
    string = file.read()
    file.close()
    
    # https://stackoverflow.com/questions/11026959/writing-a-dict-to-txt-file-and-reading-it-back
    component = eval(string)
    
    # Unfortunately, the trivial element "W(1)"" is saved in .txt as "1".
    # So, after loading from .txt all its occurrences should be coerced back tp W(1)
    
    for k in component:
        for w in component[k]:
            if w == 1:
                component[k][W(1)] = component[k][Integer(1)]
                (component[k]).pop(Integer(1))
    
    return component


def save_all(kind):
    '''Saves to external files all standard object of the given kind = "Delta", "P", "T" or "thetaL".
    See function "save".
    In case thetaL, simple modules (i.e. theta_e(L(x))) are skipped.'''
    
    total = len(W)

    i=0
    print("Started: " + str(datetime.datetime.now()))  
    
    if kind != "thetaL":

        for w in W:
            if is_saved("%s(%s)"%(kind,w), option1="only_bool") == False:

                if kind == "Delta":
                    save_(char_Delta(w))

                if kind == "P":
                    save_(char_P(w))

                if kind == "T":
                    save_(char_T(w))

            i+=1
            print(CartanType(W)[0]+str(CartanType(W)[1]) + " " + kind + ": " + str(i)+"/%d -"%total + str(datetime.datetime.now()))

    if kind == "thetaL":
        
        CT = CartanType(W)[0]+str(CartanType(W)[1])
        
        if not is_order_saved():
            print("Order not saved. Saving order first...")
            save_order()
            
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CT + "_right_order.txt"
        path = folder + file
        if os.path.isfile(path):
            f = open(path, "r")
            R_order_string = f.read()
            f.close()
        print("Order loaded.")
    
        def le(x):
            return x.length()

        W_copy = list(W)
        W_copy.sort(key=le)    
        print("W sorted.")
    
        i = 0

        for x in W_copy:
            if x != e:
                for y in W_copy:
                    if "(%s, %s)"%(convert_to_123(x),convert_to_123(y.inverse())) in R_order_string:
                        if not is_saved( "theta_%s(L(%s))"%(x,y) , "only_bool"):
                            save_(theta(x,char_L(y)))
                            print(CT + ": saved theta_%s(L(%s))"%(x,y))
                            print("%d/%d - "%(i,total) + str(datetime.datetime.now()))
            i += 1
            print("*"*20 + "\n%d/%d: x=%s done!\n"%(i,total,convert_to_123(x)) + "*"*20)
          
    print ("Finished with %s's!"%kind)
    return
    
    
    
######################## KL cells ###############################


def DR(w):
    '''Returns the set of simple right descents of w.'''
    
    return {W.simple_reflections()[i] for i in w.descents()}


def DL(w):
    '''Returns the set of simple left   descents of w.'''
        
    return DR(w.inverse())


def AL(w):
    '''Returns the set of simple left ascends of w.'''
    
    DescLe = list(DL(w))
    AscLe = [s for s in W.simple_reflections() if s not in DescLe]
    return set(AscLe)


def AR(w):
    '''Returns the set of simple right ascends of w.'''
    
    DescRi = list(DR(w))
    AscRi = [s for s in W.simple_reflections() if s not in DescRi]
    return set(AscRi)



######### In type A only:


def check_if_type_A():
    '''Used to block usage of RS correspondence outside od type A.'''
    
    if CartanType(W)[0] != 'A':
        raise ValueError("This function uses the RS correspondence, hence works only in type A!")
    

def to_perm(w):
    '''Converts an element from W to a permutation of type list.
    Mind that it is inverted in the proces.'''
    
    check_if_type_A()
    
    if w==1: w=W(1)
        
    return list((w.inverse()).to_permutation())


def to_W(t):
    '''The inverse of "to_perm".'''
    
    check_if_type_A()
    
    t = Permutation(list(t))
    red = prod(W.simple_reflections()[i] for i in t.reduced_word())
    
    if red==1: red=W(1)
    red = red.inverse()
    return red


######### In any type, but faster in A:

def KL_graph_onesided(side):
    '''Here side = "left" or "right".
    Returns a directed graph, which is almost exacly the left colored Kazhdan-Lusztig graph,
    reference: Björner-Brenti p. 176,
    but without loops and without labels.
    Used later to give preorders, and cells in types other than A.'''
    
    if side in ["l", "left", "L", "Left"]:
        side = 0
        
    if side in ["r", "right", "R", "Right"]:
        side = 1
        
    L = {}

    for x in W:
        L[x] = []
        lx = x.length()

        if side == 0:       # Björner-Brenti Lemma 6.2.4.
            for s in DL(x):
                L[x].append(s*x)

        if side == 1:
            for s in DR(x):
                L[x].append(x*s)

        for y in W.bruhat_interval(x,w0):
            ly = y.length()

            if (ly-lx)%2==1:   # Björner-Brenti Lemma 6.2.2.(i)


                if (side == 0 and DL(x).difference(DL(y)) != set()) or (side == 1 and DR(x).difference(DR(y)) != set()):
                    if mu(x,y) != 0:

                        L[x].append(y)

    return DiGraph(L)
    

def KL_graph(side):
    '''Here side = "left" or "right" or "twosided".
    Returns a directed graph, which is almost exacly the left colored Kazhdan-Lusztig graph,
    reference: Björner-Brenti p. 176,
    but without loops and without labels.
    Used later to give preorders, and cells in types other than A.'''
    
    if side in ["l", "left", "L", "Left"]:
        return KL_graph_onesided("left")
        
    if side in ["r", "right", "R", "Right"]:
        return KL_graph_onesided("right")
    
    L = KL_graph_onesided("left")    
    R = KL_graph_onesided("right")
    
    L.add_edges(R.edges())          # In two-sided case, we add all edges from left and right KL graph.
    
    return L     
    

def cell(side, w):
    '''Returns the left- or right- or 2-sided- KL cell of w.
    Use cell("left", w) for the left, cell("right", w) for the right cell,
    and cell("twoside", w) for the two-sided cell.'''
    
    if CartanType(W)[0] == 'A':   # In type A we use the RS-correspondence.
    
        RS_index = 2 # Unless said otherwise, we work with 2-sided cells.

        if side in ["left", "l", "Left", "L"]:
            RS_index = 0
        if side in ["right", "r", "Right", "R"]:
            RS_index = 1

        w = to_perm(w)  # Convert to permutation, to apply.

        lis = []

        if RS_index in [0,1]: # If we are dealing with 1-sided cells.

            for x in Permutations(range(1,n+2)):
                if RSK(x)[RS_index] == RSK(w)[RS_index]:
                    lis.append(Permutation(x))

            lis_W = [to_W(x) for x in lis]  # Convert back to W.

            return lis_W

        # Now we are dealing with 2-sided cells.

        for x in Permutations(range(1,n+2)):
            if RSK(x)[0].shape() == RSK(w)[0].shape():
                lis.append(Permutation(x))

        lis_W = [to_W(x) for x in lis]  # Convert back to W.

        return lis_W
    
    return KL_graph(side).strongly_connected_component_containing_vertex(w)   # In types other than A we use way slower procedure.
    
    
def are_cells_saved():
    '''Return "True" iff left cells are saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_left_cells.txt"
    path = folder + file
    return os.path.isfile(path)


def L_cell(w):
    '''Returns the left-KL cell of w.'''
    
    if w==e:
        return [e]
    
    if are_cells_saved():
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_left_cells.txt"
        path = folder + file
        f = open(path, "r")
        contents = f.read()
        f.close()
        LC_123 = eval(contents)
                
        w_123 = eval(convert_to_123(w))
             
        for cel in LC_123:
            if w_123 in cel:
                return [convert_from_123(str(x)) for x in cel]
            
    return cell("left", w)
    

def R_cell(w):
    '''Returns the right-KL cell of w.'''
    
    return [x.inverse() for x in L_cell(w.inverse())]


def two_cell(w):
    '''Returns the 2-sided-KL cell of w.'''
    
    Lw = L_cell(w)
    result = []
    
    for x in Lw:
        Rx = R_cell(x)
        for y in Rx:
            if y not in result:
                result.append(y)
    return result    


def cells(side):
    '''Returns the list of all left, right or two-sided cells, depending on the argument side.'''
    
    if CartanType(W)[0] == 'A':   # In type A we use the RS-correspondence.
    
        remainder = set(W)
        partition = []

        while len(remainder) != 0:

            w = remainder.pop()

            cellw = cell(side, w)

            remainder = remainder.difference(set(cellw))
            partition.append(cellw)

        return partition
    
    return KL_graph(side).strongly_connected_components()   # In types other than A we use way slower procedure.
    

def save_cells():
    '''Calculates and saves left cells.'''
    
    print("Cells already saved: %s"%are_cells_saved())
    CL = cells("left")

    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)

    result = [set( eval_number(convert_to_123(x)) for x in C)  for C in CL]
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_left_cells.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Cells now saved: %s"%are_cells_saved())
    return


def cells_graph(side):
    '''Returns directed graph where nodes are side-cells, and arrows are side-order,
    where side = "left", "right" or "twoside".'''
    
    return KL_graph(side).strongly_connected_components_digraph()  
    

def two_smaller(w,v):
    '''Checks if w is smaller than or equall to v in the two sided order.
    Uses RS-correspondence, therefore works only in type A.'''
    
    check_if_type_A()
    
    w_partition = RSK(to_perm(w))[0].shape()
    v_partition = RSK(to_perm(v))[0].shape()
    
    return w_partition.dominates(v_partition)



def is_order_saved():
    '''Return "True" iff right order is saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
    path = folder + file
    return os.path.isfile(path)


def L_smaller(x,y):
    '''Checks whether x is left-smaller than or left-equivalent to y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
    
    if x == e or y == w0:
        return True
    
    if (x == w0 and y != w0) or (y == e and x != e):
        return False
    
    if is_order_saved(): # If it is already saved:
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
        path = folder + file

        f = open(path, "r")
        R_order_string = f.read()
        f.close()  
        if "(%s, %s)"%(convert_to_123(x.inverse()),convert_to_123(y.inverse())) in R_order_string:
            return True
        else:
            return False    
       
    Lcell_x = set(cell("left",x))
    Lcell_y = set(cell("left",y))
    
    if Lcell_x == Lcell_y:
        return True

    x_less_y = False

    for path in all_paths:
        
        if path[0] == Lcell_y and path[-1] == Lcell_x:
            x_less_y = True
            break
            
    return x_less_y


def strictly_L_smaller(x,y):
    '''Checks whether x is strictly left-smaller than y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''

    if x == e:
        if y == e:
            return False
        else:
            return True
    if y == w0:
        if x == w0:
            return False
        else:
            return True        
        
    if is_order_saved(): # If it is already saved:
        
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
        path = folder + file

        if L_smaller(x,y) and not L_smaller(y,x):
            return True
        else:
            return False    
    
    Lcell_x = set(cell("left",x))
    Lcell_y = set(cell("left",y))

    if Lcell_x == Lcell_y:
        return False
    
    x_less_y = False

    for path in all_paths:
        
        if path[0] == Lcell_y and path[-1] == Lcell_x:
            x_less_y = True
            break
            
    return x_less_y


def R_smaller(x,y):
    '''Checks whether x is right-smaller than or right-equivalent to y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
       
    return L_smaller(x.inverse(),y.inverse())


def strictly_R_smaller(x,y):
    '''Checks whether x is strictly left-smaller than y, for x,y in W.
    For this the line "all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
    
    return strictly_L_smaller(x.inverse(),y.inverse())


def save_order():
    '''Calculates and saves right order.'''
    
    print("Order already saved: %s"%is_order_saved())
    
    # This could take forever to calculate. There probably exists more efficient way.
    all_paths = cells_graph("left").all_simple_paths(trivial=True) 

    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)

    result = set()

    for path in all_paths:
        Y = path[0]
        X = path[-1]
        for x in X:
            for y in Y:
                result.add( (eval_number(convert_to_123(x.inverse())),eval_number(convert_to_123(y.inverse()))) )
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_right_order.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Order now saved: %s"%is_order_saved())
    return

    
######################## Lusztig's a function ###############################


def is_involution(w):
    '''Checks if w is an involution.'''
    
    return w == w.inverse()


def Duflo_involution(w, side):
    '''Returns the unique (Duflo) involution from the side="left" or "right" cell of w.'''
        
    if CartanType(W)[0] == 'A':
    
        cellw = cell(side,w)

        for x in cellw:

            if is_involution(x):
                break

        return x
    
    # Much slower for other types, loads "L_cell()" and "Duflo_Involutions()".
    if side == "right":
        return list(set(R_cell(w)).intersection(Duflo_Involutions()))[0]
    
    if side == "left":
        return list(set(L_cell(w)).intersection(Duflo_Involutions()))[0]

    
def Duflo_Involutions():
    '''Returns the list of Duflo involutions.'''
     
    # If it is already saved
    if are_Duflo_involutions_saved():
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_Duflo_involutions.txt"
        path = folder + file
        f = open(path, "r")
        contents = f.read()
        f.close()
        D = [convert_from_123(str(x)) for x in eval(contents)]
        return D    

    Involutions = {x for x in W if is_involution(x)}
    
    if CartanType(W)[0] == 'A':
        return list(Involutions)  
    
    lis = []

    for cell in cells("l"):

        a = w0.length()
        cand = w0

        for x in (set(cell)).intersection(Involutions):

            ax = earliest_occurence(x)

            if ax < a:
                a = ax
                cand = x

        lis.append(cand)
    
    return lis


def are_Duflo_involutions_saved():
    '''Return "True" iff Duflo involutions are saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_Duflo_involutions.txt"
    path = folder + file
    return os.path.isfile(path)


def save_Duflo_involutions():
    '''Calculates and saves Duflo involutions.'''
    
    print("Duflo involutions already saved: %s"%are_Duflo_involutions_saved())
    D = Duflo_Involutions()
    
    def l(x):   # Needed to sort D
        return x.length()
    D.sort(key = l)
    
    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)
    
    result = [eval_number(convert_to_123(x)) for x in D]
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_Duflo_involutions.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Duflo involutions now saved: %s"%are_Duflo_involutions_saved())
    return


def a(w):
    '''Returns the value of Lusztig's a function on w.
    Much faster in type A.'''
    
    if CartanType(W)[0] == 'A':
    
        if is_involution(w):
            return earliest_occurence(w)
    
    d = Duflo_involution(w,"right")
    
    return earliest_occurence(d)
    


def b(x,y):
    '''Returns the function b(x,y) from "SHPOV".'''
    
    if R_smaller(x,y.inverse()):
        return max(components( theta(x,char_L(y)) ))
    else:
        return -1 # Or some very negative number.


################## Delta flags ##################

def Delta_flag(X):
    '''Returns a dicionary where keys are graded degrees, and values are dictionaries
    where keys are parameters of Deltas and values are multiplicities in the given degree.'''
    
    if X == char_0():
        return {}
    
    if not X.is_true_character():
        raise TypeError("Does not have a Delta flag!") 
    

    result = {}  
    X_ = deepcopy(X)
    top_degree = min(components(X))
    top = deepcopy(X)
    truncate(top, [i for i in components(X) if i != top_degree] )   
    result.update(top.component)
    
    
    for w in top.component[top_degree]:
        X_ -=   shift(top.component[top_degree][w] * char_Delta(w), -top_degree)
    X_.remove_zeros()
    
    result.update(Delta_flag(X_))
    
    return result


def print_Delta_flag(X):
    '''Prints out the parameters of Delta flag of a graded character X.'''
    
    result = char_0()
    result.component = Delta_flag(X)
    result.name = "Delta flag of %s"%X.name
    print_clean(result)




print(DynkinDiagram(W))
    

    


        O 5
        |
        |
O---O---O---O
1   2   3   4   
D5


In [6]:
def br_le(x,y):
    return x.bruhat_le(y)  


def W_subposet(L):
    '''This is equivalent to W.bruhat_poset().subposet(),
    but without calculating the full W.bruhat_poset().'''
      
    return Poset((L,br_le))


def join_old(S):
    SS = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]
    
    U = set(W.bruhat_interval(SS[0],w0))
    for a in SS[1:]:
        U = U.intersection(set(W.bruhat_interval(a,w0)))
        
    minU = (W_subposet(list(U))).minimal_elements()
    
    return minU


def join(S):
    S = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]   

    ld = set()
    rd = set()
    for s in S:
        ld = ld.union(DL(s))
        rd = rd.union(DR(s))

    U = []
    for y in [x for x in W if DL(x).issubset(ld) and  DR(x).issubset(rd)]:
        is_upper = True
        for s in S:
            if not s.bruhat_le(y):
                is_upper = False
        if is_upper == True:
            U.append(y)
    
    return (W_subposet(U)).minimal_elements()      
    
    
def JM_(w):
    if w not in W:
        w = convert_from_123(w)
    result = W_subposet([x for x in join_irreducibles if x.bruhat_le(w) ]).maximal_elements()
    return [convert_from_123(convert_to_123(x)) for x in result]

def BM(w):
    if w not in W:
        w = convert_from_123(w)
    return W_subposet([x for x in bigrassmannians if x.bruhat_le(w) ]).maximal_elements()

def JM__(w):
    if w not in W:
        w = convert_from_123(w)
    DLw = DL(w)
    DRw = DR(w)
    result = []
    for x in JM_(w):
        x = convert_from_123(convert_to_123(x))
        if list(DL(x))[0] in DLw and list(DR(x))[0] in DRw:
            result.append(x)
    return result

def JM(w):
    if w not in W:
        w = convert_from_123(w)
    DLw = DL(w)
    DRw = DR(w)
    result = W_subposet([x for x in join_irreducibles if x.bruhat_le(w) and (list(DL(x))[0] in DLw) and (list(DR(x))[0] in DRw) ]).maximal_elements()
    return [convert_from_123(convert_to_123(x)) for x in result]    

In [37]:
######################## A ########################

bigrassmannians = []
for x in W:
    if len(DL(x))==1 and len(DR(x))==1:
        bigrassmannians.append(x)

join_irreducibles = [x for x in bigrassmannians]


In [74]:
######################## B2 ########################

#sum_J KL = 6

#bigrassmannians = join_irreducibles = dissectors: 6

bigrassmannians = [s2, s1, s1*s2*s1, s1*s2, s2*s1, s2*s1*s2]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]

In [79]:
######################## G2 ########################

#sum_J KL = 10

#bigrassmannians = join_irreducibles = dissectors: 10

bigrassmannians = [s2*s1*s2*s1, s1*s2*s1*s2, s2, s2*s1*s2*s1*s2, s1*s2*s1, s1, s1*s2*s1*s2*s1, s2*s1*s2, s1*s2, s2*s1]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]

In [19]:
######################## B3 ########################

#sum_J KL = 19

#bigrassmannians = join_irreducibles = dissectors: 19

bigrassmannians = [s3, s3*s2, s3*s2*s1, s3*s2*s3, s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s2, s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s2, s1*s2*s3*s2*s1]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]


In [27]:
######################## B4 ########################

#sum_J KL = 44

#bigrassmannians: 45

bigrassmannians = [s2*s3*s1*s2, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s1, s2*s3, s1*s2, s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s1, s3, s1*s2*s3, s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s4, s2, s3*s4*s2*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s4*s3*s4, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s4*s3*s4*s2*s3, s1*s2*s3*s4*s3*s2*s1, s4*s3*s4*s2*s3*s4*s1*s2*s3, s3*s4*s3, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s2*s3*s4*s3*s2, s4, s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s3*s4, s4*s3*s2, s1*s2*s3*s4*s3*s2, s4*s3, s4*s3*s4*s2*s3*s4, s1*s2*s3*s4, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s2*s3*s4*s3*s2*s1, s2*s3*s4*s1*s2*s3, s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s3*s1*s2, s3*s4*s2*s3, s2*s3*s4*s3, s3*s4*s3*s2]

#join_irreducibles = dissectors: 44

join_irreducibles = [s4, s3, s2, s1, s3*s2*s1, s1*s2*s3, s2*s3*s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s3*s2*s1, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s4*s3*s4*s2*s3*s4, s2*s3*s1*s2, s3*s4*s2*s3, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s3*s4*s3, s4*s3*s4*s2*s3, s2*s1, s3*s4*s2*s3*s4, s1*s2, s2*s3*s4*s3, s4*s3*s2*s1, s3*s4*s3*s2, s1*s2*s3*s4, s2*s3*s4*s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s2*s3, s1*s2*s3*s4*s3*s2, s2*s3*s4*s1*s2*s3, s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s2*s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s4*s3*s4, s3*s4*s2*s3*s4*s2*s3, s4*s3, s3*s4*s2*s3*s4*s1*s2*s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3]

dissectors = [x for x in join_irreducibles]

In [43]:
######################## D4 ########################

#sum_J KL = 28

#bigrassmannians: 30

bigrassmannians = [s4*s2*s3*s1*s2*s4, s2*s4*s1*s2, s2*s3*s1*s2, s2*s4*s3*s2, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3, s2*s4*s3*s1*s2]

#join_irreducibles: 29

join_irreducibles = [s3, s4, s2, s1, s2*s4*s3*s2, s3*s2*s1, s2*s4*s1*s2, s1*s2*s3, s4*s2*s1, s1*s2*s4, s3*s2*s4, s1*s2, s2*s4, s3*s2*s4*s1*s2*s3, s1*s2*s4*s3*s2*s1, s4*s2*s3, s3*s2, s4*s2, s4*s2*s3*s1*s2*s4, s1*s2*s4*s3*s2, s2*s1, s4*s2*s3*s1*s2, s3*s2*s4*s1*s2, s2*s4*s1*s2*s3, s2*s3*s1*s2, s2*s3*s1*s2*s4, s2*s4*s3*s2*s1, s2*s3, s2*s3*s1*s2*s4*s2*s3*s1*s2]

#dissectors: 26

dissectors = [s4*s2*s3*s1*s2*s4, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3]


In [35]:
######################## F4 ########################

#sum_J KL = 92

#bigrassmannians: 108

bigrassmannians = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]

#join_irreducibles: 96

join_irreducibles = [s3, s2, s1, s4, s3*s4, s2*s3, s1*s2, s2*s3*s2, s3*s4*s2*s3, s2*s3*s2*s1, s1*s2*s3, s1*s2*s3*s2, s1*s2*s3*s4, s3*s4*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3, s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3, s2*s3*s4, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4, s4*s3*s2, s4*s3*s2*s3, s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s4*s3*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2, s3*s2*s3, s3*s4*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s1, s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2]

#dissectors: 80

dissectors = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]


In [3]:
######################## B5 ########################

#sum_J KL = 85

#bigrassmannians: 90

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s2*s3, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

#join_irreducibles = dissectors: 85

join_irreducibles = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

dissectors = [x for x in join_irreducibles]

In [7]:
######################## D5 ########################

#sum_J KL = 60

#bigrassmannians: 69

bigrassmannians = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s3*s4*s2*s3, s3*s5*s2*s3, s3*s5*s4*s3, s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s4*s2*s3*s5, s3*s5*s4*s2*s3, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2, s3*s5*s4*s3*s2*s1, s3*s5*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s3*s4*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3*s4, s2*s3*s5*s4*s3*s1*s2]

#join_irreducibles: 64

join_irreducibles = [s3, s2, s4, s5, s1, s2*s3, s4*s3, s5*s3, s1*s2, s2*s3*s4, s4*s3*s2, s2*s3*s5, s5*s3*s2, s4*s3*s5, s5*s3*s4, s4*s3*s2*s1, s1*s2*s3, s5*s3*s2*s1, s3*s2, s3*s2*s1, s1*s2*s3*s4, s1*s2*s3*s5, s3*s4, s3*s4*s2*s3, s3*s4*s2*s3*s1*s2, s3*s5, s3*s5*s2*s3, s3*s5*s2*s3*s1*s2, s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s5*s4*s3*s2, s3*s5*s2*s3*s4, s3*s4*s2*s3*s5, s2*s3*s5*s4*s3*s2, s2*s3*s5*s4*s3, s3*s5*s4*s3*s2*s1, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s5*s4*s3*s2*s1, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5]


#dissectors: 52

dissectors = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s1, s2*s3*s1*s2, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4]


In [17]:
######################## B6 ########################

#sum_J KL = 146

#bigrassmannians: 161

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s6, s3*s4*s5*s6*s5, s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s5*s4, s3*s4*s5*s6*s5*s4*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s5, s2*s3*s4*s5*s6*s5*s4, s2*s3*s4*s5*s6*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6, s4*s5*s6*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s5, s4*s5*s6*s5*s3*s4, s4*s5*s6*s5*s3*s4*s2*s3, s4*s5*s6*s3*s4*s5*s6*s4*s5, s4*s5*s6*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s5*s4, s4*s5*s6*s5*s4*s3, s4*s5*s6*s5*s4*s3*s2, s4*s5*s6*s5*s4*s3*s2*s1, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s6, s1*s2*s3*s4*s5*s6*s5, s1*s2*s3*s4*s5*s6*s5*s4, s1*s2*s3*s4*s5*s6*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s6, s5*s6*s4*s5, s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s5*s6*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s5, s5*s6*s5*s4, s5*s6*s5*s4*s3, s5*s6*s5*s4*s3*s2, s5*s6*s5*s4*s3*s2*s1, s5*s6*s4*s5*s6*s4*s5, s5*s6*s4*s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5, s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s2*s3*s4*s5*s6*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s5*s4*s1*s2*s3, s2*s3*s4*s5*s6*s5*s4*s3*s1*s2, s6, s6*s5, s6*s5*s4, s6*s5*s4*s3, s6*s5*s4*s3*s2, s6*s5*s4*s3*s2*s1, s6*s5*s6, s6*s5*s6*s4*s5, s6*s5*s6*s4*s5*s3*s4, s6*s5*s6*s4*s5*s3*s4*s2*s3, s6*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s5*s6*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5, s6*s5*s6*s4*s5*s6*s3*s4*s5*s6, s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s5*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s5*s6*s5*s4*s3*s2, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s1*s2*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s4*s5*s2*s3*s4, s4*s5*s6*s3*s4*s5*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s5*s4*s3*s2, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4, s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5, s3*s4*s5*s6*s2*s3*s4*s5*s6, s3*s4*s5*s6*s2*s3*s4*s5, s3*s4*s5*s6*s5*s2*s3*s4, s3*s4*s5*s6*s5*s4*s2*s3, s3*s4*s5*s6*s5*s4*s3*s2, s3*s4*s5*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s5*s4*s2*s3*s1*s2, s3*s4*s5*s6*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4]

#join_irreducibles = dissectors: 146

join_irreducibles = [s6,  s5,  s4,  s3,  s2,  s1,  s6*s5,  s5*s6,  s5*s4,  s4*s5,  s4*s3,  s3*s4,  s3*s2,  s2*s3,  s2*s1,  s1*s2,  s6*s5*s6,  s6*s5*s4,  s5*s6*s5,  s5*s4*s3,  s4*s5*s6,  s4*s3*s2,  s3*s4*s5,  s3*s2*s1,  s2*s3*s4,  s1*s2*s3,  s6*s5*s4*s3,  s5*s6*s4*s5,  s5*s6*s5*s4,  s5*s4*s3*s2,  s4*s5*s6*s5,  s4*s5*s3*s4,  s4*s3*s2*s1,  s3*s4*s5*s6,  s3*s4*s2*s3,  s2*s3*s4*s5,  s2*s3*s1*s2,  s1*s2*s3*s4,  s6*s5*s6*s4*s5,  s6*s5*s4*s3*s2,  s5*s6*s4*s5*s6,  s5*s6*s5*s4*s3,  s5*s4*s3*s2*s1,  s4*s5*s6*s5*s4,  s3*s4*s5*s6*s5,  s2*s3*s4*s5*s6,  s1*s2*s3*s4*s5,  s6*s5*s6*s4*s5*s6,  s6*s5*s4*s3*s2*s1,  s5*s6*s4*s5*s3*s4,  s5*s6*s5*s4*s3*s2,  s4*s5*s6*s3*s4*s5,  s4*s5*s6*s5*s4*s3,  s4*s5*s3*s4*s2*s3,  s3*s4*s5*s6*s5*s4,  s3*s4*s5*s2*s3*s4,  s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s5,  s2*s3*s4*s1*s2*s3,  s1*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s3*s4,  s5*s6*s4*s5*s6*s4*s5,  s5*s6*s5*s4*s3*s2*s1,  s4*s5*s6*s3*s4*s5*s6,  s4*s5*s6*s5*s4*s3*s2,  s3*s4*s5*s6*s5*s4*s3,  s2*s3*s4*s5*s6*s5*s4,  s1*s2*s3*s4*s5*s6*s5,  s5*s6*s4*s5*s6*s3*s4*s5,  s5*s6*s4*s5*s3*s4*s2*s3,  s4*s5*s3*s4*s2*s3*s1*s2,  s4*s5*s6*s5*s4*s3*s2*s1,  s3*s4*s5*s6*s2*s3*s4*s5,  s3*s4*s5*s6*s5*s4*s3*s2,  s2*s3*s4*s5*s1*s2*s3*s4,  s2*s3*s4*s5*s6*s5*s4*s3,  s1*s2*s3*s4*s5*s6*s5*s4,  s6*s5*s6*s4*s5*s6*s3*s4*s5,  s6*s5*s6*s4*s5*s3*s4*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6,  s5*s6*s4*s5*s6*s4*s5*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s4*s5,  s4*s5*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6,  s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s5*s4*s3*s2*s1,  s2*s3*s4*s5*s6*s5*s4*s3*s2,  s1*s2*s3*s4*s5*s6*s5*s4*s3,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6,  s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s5*s6*s5*s4*s3*s2*s1,  s1*s2*s3*s4*s5*s6*s5*s4*s3*s2,  s6*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s1*s2*s3*s4*s5*s6*s5*s4*s3*s2*s1,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s6*s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5,  s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s5*s6*s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5,  s4*s5*s6*s3*s4*s5*s6*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4]

dissectors = [x for x in join_irreducibles]

In [56]:
######################## D6 ########################

#sum_J KL = 110

#bigrassmannians: 135

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s6, s3*s4*s2*s3, s3*s4*s6*s5*s4, s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4, s3*s4*s6*s5*s4*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s6, s2*s3*s4*s6*s5*s4, s2*s3*s4*s6*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s6, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s4*s6, s4*s6*s3*s4, s4*s6*s3*s4*s5, s4*s6*s3*s4*s2*s3, s4*s6*s3*s4*s2*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s5*s3*s4, s4*s6*s5*s3*s4*s2*s3, s4*s6*s5*s3*s4*s2*s3*s1*s2, s4*s6*s5*s4, s4*s6*s5*s4*s3, s4*s6*s5*s4*s3*s2, s4*s6*s5*s4*s3*s2*s1, s4*s5*s3*s4*s6*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s6, s1*s2*s3*s4*s6*s5*s4, s1*s2*s3*s4*s6*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s6, s5*s4*s6*s3*s4, s5*s4*s6*s3*s4*s5, s5*s4*s6*s3*s4*s2*s3, s5*s4*s6*s3*s4*s2*s3*s1*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6, s6*s4, s6*s4*s3, s6*s4*s3*s2, s6*s4*s3*s2*s1, s6*s4*s5, s6*s4*s5*s3*s4, s6*s4*s5*s3*s4*s6, s6*s4*s5*s3*s4*s2*s3, s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s4*s5*s3*s4*s6*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s6*s5*s4*s3*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4*s5, s2*s3*s4*s6*s5*s4*s1*s2*s3, s2*s3*s4*s6*s5*s4*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s3*s4*s5*s2*s3*s4*s6, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s6, s3*s4*s6*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5, s3*s4*s6*s5*s4*s2*s3, s3*s4*s6*s5*s4*s3*s2, s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s6*s5*s4*s2*s3*s1*s2, s3*s4*s6*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6]

#join_irreducibles: 120

join_irreducibles = [s6,  s5,  s4,  s3,  s2,  s1,  s6*s4,  s5*s4,  s4*s6,  s4*s5,  s4*s3,  s3*s4,  s3*s2,  s2*s3,  s2*s1,  s1*s2,  s6*s4*s5,  s6*s4*s3,  s5*s4*s6,  s5*s4*s3,  s4*s3*s2,  s3*s4*s6,  s3*s4*s5,  s3*s2*s1,  s2*s3*s4,  s1*s2*s3,  s6*s4*s3*s2,  s5*s4*s3*s2,  s4*s6*s5*s4,  s4*s6*s3*s4,  s4*s5*s3*s4,  s4*s3*s2*s1,  s3*s4*s2*s3,  s2*s3*s4*s6,  s2*s3*s4*s5,  s2*s3*s1*s2,  s1*s2*s3*s4,  s6*s4*s5*s3*s4,  s6*s4*s3*s2*s1,  s5*s4*s6*s3*s4,  s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6,  s4*s6*s3*s4*s5,  s4*s6*s5*s4*s3,  s3*s4*s6*s5*s4,  s1*s2*s3*s4*s6,  s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6,  s5*s4*s6*s3*s4*s5,  s4*s6*s3*s4*s2*s3,  s4*s5*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2,  s3*s4*s6*s2*s3*s4,  s3*s4*s5*s2*s3*s4,  s3*s4*s6*s5*s4*s3,  s3*s4*s2*s3*s1*s2,  s2*s3*s4*s6*s5*s4,  s2*s3*s4*s1*s2*s3,  s6*s4*s5*s3*s4*s2*s3,  s5*s4*s6*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2*s1,  s3*s4*s5*s2*s3*s4*s6,  s3*s4*s6*s2*s3*s4*s5,  s3*s4*s6*s5*s4*s3*s2,  s2*s3*s4*s6*s5*s4*s3,  s1*s2*s3*s4*s6*s5*s4,  s4*s6*s3*s4*s5*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4,  s4*s6*s3*s4*s2*s3*s1*s2,  s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s6*s5*s4*s3*s2*s1,  s2*s3*s4*s6*s1*s2*s3*s4,  s2*s3*s4*s5*s1*s2*s3*s4,  s2*s3*s4*s6*s5*s4*s3*s2,  s1*s2*s3*s4*s6*s5*s4*s3,  s6*s4*s5*s3*s4*s6*s2*s3*s4,  s6*s4*s5*s3*s4*s2*s3*s1*s2,  s5*s4*s6*s3*s4*s5*s2*s3*s4,  s5*s4*s6*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6,  s4*s5*s3*s4*s6*s2*s3*s4*s5,  s4*s5*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s1*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6,  s2*s3*s4*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s6*s5*s4*s3*s2*s1,  s1*s2*s3*s4*s6*s5*s4*s3*s2,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6,  s1*s2*s3*s4*s6*s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4]

#dissectors: ?

In [2]:
######################## E6 ########################

#sum_J KL = 156

#bigrassmannians: 232

bigrassmannians = [s6, s6*s5, s6*s5*s4, s6*s5*s4*s2, s6*s5*s4*s3, s6*s5*s4*s3*s1, s6*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5, s5*s6, s5*s4, s5*s6*s4*s5, s5*s4*s2, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3, s5*s6*s4*s5*s2*s4*s3*s1, s5*s4*s3, s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s2, s5*s6*s4*s5*s3*s2*s4, s5*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4, s4*s5, s4*s5*s6, s4*s2, s4*s5*s2*s4, s4*s5*s6*s2*s4*s5, s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3, s4*s5*s3*s4, s4*s5*s6*s3*s4*s5, s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s1, s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4, s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2, s2*s4, s2*s4*s5, s2*s4*s5*s6, s2*s4*s3, s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3, s3*s4, s3*s4*s5, s3*s4*s5*s6, s3*s4*s2, s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s1, s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3, s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1, s1*s3, s1*s3*s4, s1*s3*s4*s5, s1*s3*s4*s5*s6, s1*s3*s4*s2, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6]

#join_irreducibles: 182

join_irreducibles = [s1, s2, s3, s4, s5, s6, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s6*s5*s4, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s5*s4*s3*s1, s5*s6*s4*s5, s6*s5*s4*s2, s6*s5*s4*s3, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s3*s4*s5*s2*s4, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s3*s2*s4*s5, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4, s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#dissectors: ?

In [23]:
######################## B9 ########################

#sum_J KL = 489

#bigrassmannians: 615

bigrassmannians = [s9, s8, s7, s6, s5, s4, s3, s2, s1, s9*s8, s8*s9, s8*s7, s7*s8, s7*s6, s6*s7, s6*s5, s5*s6, s5*s4, s4*s5, s4*s3, s3*s4, s3*s2, s2*s3, s2*s1, s1*s2, s9*s8*s9, s9*s8*s7, s8*s9*s8, s8*s7*s6, s7*s8*s9, s7*s6*s5, s6*s7*s8, s6*s5*s4, s5*s6*s7, s5*s4*s3, s4*s5*s6, s4*s3*s2, s3*s4*s5, s3*s2*s1, s2*s3*s4, s1*s2*s3, s9*s8*s7*s6, s8*s9*s7*s8, s8*s9*s8*s7, s8*s7*s6*s5, s7*s8*s9*s8, s7*s8*s6*s7, s7*s6*s5*s4, s6*s7*s8*s9, s6*s7*s5*s6, s6*s5*s4*s3, s5*s6*s7*s8, s5*s6*s4*s5, s5*s4*s3*s2, s4*s5*s6*s7, s4*s5*s3*s4, s4*s3*s2*s1, s3*s4*s5*s6, s3*s4*s2*s3, s2*s3*s4*s5, s2*s3*s1*s2, s1*s2*s3*s4, s9*s8*s9*s7*s8, s9*s8*s7*s6*s5, s8*s9*s7*s8*s9, s8*s9*s8*s7*s6, s8*s7*s6*s5*s4, s7*s8*s9*s8*s7, s7*s6*s5*s4*s3, s6*s7*s8*s9*s8, s6*s5*s4*s3*s2, s5*s6*s7*s8*s9, s5*s4*s3*s2*s1, s4*s5*s6*s7*s8, s3*s4*s5*s6*s7, s2*s3*s4*s5*s6, s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9, s9*s8*s7*s6*s5*s4, s8*s9*s7*s8*s6*s7, s8*s9*s8*s7*s6*s5, s8*s7*s6*s5*s4*s3, s7*s8*s9*s6*s7*s8, s7*s8*s9*s8*s6*s7, s7*s8*s9*s8*s7*s6, s7*s8*s6*s7*s5*s6, s7*s6*s5*s4*s3*s2, s6*s7*s8*s9*s8*s7, s6*s7*s8*s5*s6*s7, s6*s7*s5*s6*s4*s5, s6*s5*s4*s3*s2*s1, s5*s6*s7*s8*s9*s8, s5*s6*s7*s4*s5*s6, s5*s6*s4*s5*s3*s4, s4*s5*s6*s7*s8*s9, s4*s5*s6*s3*s4*s5, s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s7*s8, s3*s4*s5*s2*s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s7, s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s5*s6, s9*s8*s9*s7*s8*s6*s7, s9*s8*s7*s6*s5*s4*s3, s8*s9*s7*s8*s9*s7*s8, s8*s9*s8*s7*s6*s5*s4, s8*s7*s6*s5*s4*s3*s2, s7*s8*s9*s6*s7*s8*s9, s7*s8*s9*s8*s7*s6*s5, s7*s6*s5*s4*s3*s2*s1, s6*s7*s8*s9*s8*s7*s6, s5*s6*s7*s8*s9*s8*s7, s4*s5*s6*s7*s8*s9*s8, s3*s4*s5*s6*s7*s8*s9, s2*s3*s4*s5*s6*s7*s8, s1*s2*s3*s4*s5*s6*s7, s9*s8*s7*s6*s5*s4*s3*s2, s8*s9*s7*s8*s9*s6*s7*s8, s8*s9*s7*s8*s6*s7*s5*s6, s8*s9*s8*s7*s6*s5*s4*s3, s8*s7*s6*s5*s4*s3*s2*s1, s7*s8*s9*s8*s6*s7*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s8*s7*s6*s5*s4, s6*s7*s8*s9*s5*s6*s7*s8, s6*s7*s8*s9*s8*s5*s6*s7, s6*s7*s8*s9*s8*s7*s5*s6, s6*s7*s8*s9*s8*s7*s6*s5, s6*s7*s5*s6*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7, s5*s6*s7*s8*s9*s8*s7*s6, s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s8*s7, s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s7*s8*s9*s8, s3*s4*s5*s6*s2*s3*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9, s2*s3*s4*s5*s1*s2*s3*s4, s1*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8, s9*s8*s9*s7*s8*s6*s7*s5*s6, s9*s8*s7*s6*s5*s4*s3*s2*s1, s8*s9*s7*s8*s9*s6*s7*s8*s9, s8*s9*s7*s8*s9*s7*s8*s6*s7, s8*s9*s8*s7*s6*s5*s4*s3*s2, s7*s8*s9*s6*s7*s8*s9*s7*s8, s7*s8*s9*s6*s7*s8*s5*s6*s7, s7*s8*s9*s8*s7*s6*s5*s4*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9, s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s7*s8*s9*s8*s7*s6*s5*s4, s5*s6*s7*s8*s9*s8*s7*s6*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s9*s8*s7*s6, s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s8*s7, s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s8, s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s7*s8*s9*s8*s6*s7*s5*s6*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s6*s7*s8*s9*s8*s7*s5*s6*s4*s5, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s8*s4*s5*s6*s7, s5*s6*s7*s8*s9*s8*s7*s4*s5*s6, s5*s6*s7*s8*s9*s8*s7*s6*s4*s5, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7, s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8, s6*s7*s8*s9*s8*s5*s6*s7*s4*s5*s6, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s7*s8*s5*s6*s7, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s8*s7*s5*s6*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s9*s8*s7*s6*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s4*s5*s6*s7*s8*s9*s8*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s8*s7*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s8*s7*s6*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s3*s4, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8, s5*s6*s7*s8*s9*s8*s7*s4*s5*s6*s3*s4*s5, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s7*s8*s9*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s6*s7*s8*s9*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s8*s7*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s7*s8*s9*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s9*s8*s7*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s8*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s9*s8*s7*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s2*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s2*s3, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s8*s7*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s7*s8*s9*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s6*s7*s8*s9*s8*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s7*s8*s9*s8*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s8*s7*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s7*s8*s9*s8*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s2*s3*s4*s5*s6*s7*s8*s9*s8*s1*s2*s3*s4*s5*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s1*s2*s3*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s1*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s1*s2, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s6*s7*s8*s9*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s4*s5*s6*s7*s8*s9*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s8*s7*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s2*s3*s4*s5*s1*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s5*s6*s7*s8*s9*s8*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s3*s4*s5*s6*s7*s8*s9*s8*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s8*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s2*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s1*s2*s3*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s1*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s1*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s1*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6]

#join_irreducibles = dissectors: 489

join_irreducibles = [s9, s8, s7, s6, s5, s4, s3, s2, s1, s9*s8, s8*s9, s8*s7, s7*s8, s7*s6, s6*s7, s6*s5, s5*s6, s5*s4, s4*s5, s4*s3, s3*s4, s3*s2, s2*s3, s2*s1, s1*s2, s9*s8*s9, s9*s8*s7, s8*s9*s8, s8*s7*s6, s7*s8*s9, s7*s6*s5, s6*s7*s8, s6*s5*s4, s5*s6*s7, s5*s4*s3, s4*s5*s6, s4*s3*s2, s3*s4*s5, s3*s2*s1, s2*s3*s4, s1*s2*s3, s9*s8*s7*s6, s8*s9*s7*s8, s8*s9*s8*s7, s8*s7*s6*s5, s7*s8*s9*s8, s7*s8*s6*s7, s7*s6*s5*s4, s6*s7*s8*s9, s6*s7*s5*s6, s6*s5*s4*s3, s5*s6*s7*s8, s5*s6*s4*s5, s5*s4*s3*s2, s4*s5*s6*s7, s4*s5*s3*s4, s4*s3*s2*s1, s3*s4*s5*s6, s3*s4*s2*s3, s2*s3*s4*s5, s2*s3*s1*s2, s1*s2*s3*s4, s9*s8*s9*s7*s8, s9*s8*s7*s6*s5, s8*s9*s7*s8*s9, s8*s9*s8*s7*s6, s8*s7*s6*s5*s4, s7*s8*s9*s8*s7, s7*s6*s5*s4*s3, s6*s7*s8*s9*s8, s6*s5*s4*s3*s2, s5*s6*s7*s8*s9, s5*s4*s3*s2*s1, s4*s5*s6*s7*s8, s3*s4*s5*s6*s7, s2*s3*s4*s5*s6, s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9, s9*s8*s7*s6*s5*s4, s8*s9*s7*s8*s6*s7, s8*s9*s8*s7*s6*s5, s8*s7*s6*s5*s4*s3, s7*s8*s9*s6*s7*s8, s7*s8*s9*s8*s7*s6, s7*s8*s6*s7*s5*s6, s7*s6*s5*s4*s3*s2, s6*s7*s8*s9*s8*s7, s6*s7*s8*s5*s6*s7, s6*s7*s5*s6*s4*s5, s6*s5*s4*s3*s2*s1, s5*s6*s7*s8*s9*s8, s5*s6*s7*s4*s5*s6, s5*s6*s4*s5*s3*s4, s4*s5*s6*s7*s8*s9, s4*s5*s6*s3*s4*s5, s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s7*s8, s3*s4*s5*s2*s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s7, s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s5*s6, s9*s8*s9*s7*s8*s6*s7, s9*s8*s7*s6*s5*s4*s3, s8*s9*s7*s8*s9*s7*s8, s8*s9*s8*s7*s6*s5*s4, s8*s7*s6*s5*s4*s3*s2, s7*s8*s9*s6*s7*s8*s9, s7*s8*s9*s8*s7*s6*s5, s7*s6*s5*s4*s3*s2*s1, s6*s7*s8*s9*s8*s7*s6, s5*s6*s7*s8*s9*s8*s7, s4*s5*s6*s7*s8*s9*s8, s3*s4*s5*s6*s7*s8*s9, s2*s3*s4*s5*s6*s7*s8, s1*s2*s3*s4*s5*s6*s7, s9*s8*s7*s6*s5*s4*s3*s2, s8*s9*s7*s8*s9*s6*s7*s8, s8*s9*s7*s8*s6*s7*s5*s6, s8*s9*s8*s7*s6*s5*s4*s3, s8*s7*s6*s5*s4*s3*s2*s1, s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s8*s7*s6*s5*s4, s6*s7*s8*s9*s5*s6*s7*s8, s6*s7*s8*s9*s8*s7*s6*s5, s6*s7*s5*s6*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7, s5*s6*s7*s8*s9*s8*s7*s6, s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s8*s7, s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s7*s8*s9*s8, s3*s4*s5*s6*s2*s3*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9, s2*s3*s4*s5*s1*s2*s3*s4, s1*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8, s9*s8*s9*s7*s8*s6*s7*s5*s6, s9*s8*s7*s6*s5*s4*s3*s2*s1, s8*s9*s7*s8*s9*s6*s7*s8*s9, s8*s9*s7*s8*s9*s7*s8*s6*s7, s8*s9*s8*s7*s6*s5*s4*s3*s2, s7*s8*s9*s6*s7*s8*s9*s7*s8, s7*s8*s9*s6*s7*s8*s5*s6*s7, s7*s8*s9*s8*s7*s6*s5*s4*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9, s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s7*s8*s9*s8*s7*s6*s5*s4, s5*s6*s7*s8*s9*s8*s7*s6*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s9*s8*s7*s6, s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s8*s7, s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s8, s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7, s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7, s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8, s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3, s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2, s9*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s1*s2*s3*s4*s5*s6*s7*s8*s9*s8*s7*s6*s5*s4*s3*s2*s1, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6, s9*s8*s9*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7, s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2*s3*s1*s2, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s9*s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6, s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s3*s4*s5*s1*s2*s3*s4, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7, s8*s9*s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8, s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s3*s4*s5*s6*s1*s2*s3*s4*s5, s7*s8*s9*s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s1*s2*s3*s4*s5*s6*s7, s6*s7*s8*s9*s5*s6*s7*s8*s9*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s9*s2*s3*s4*s5*s6*s7*s8*s9*s1*s2*s3*s4*s5*s6*s7*s8*s9*s3*s4*s5*s6*s7*s8*s2*s3*s4*s5*s6*s7*s1*s2*s3*s4*s5*s6]

dissectors = [x for x in join_irreducibles]

In [16]:
BG = {}
for x in bigrassmannians:
    i = eval(convert_to_123(list(DL(x))[0]))
    j = eval(convert_to_123(list(DR(x))[0]))
    if (i,j) not in BG:
        BG[(i,j)] = []  
    BG[(i,j)].append(x)
print("BG done!")

JI = {}
for x in join_irreducibles:
    i = eval(convert_to_123(list(DL(x))[0]))
    j = eval(convert_to_123(list(DR(x))[0]))
    if (i,j) not in JI:
        JI[(i,j)] = []  
    JI[(i,j)].append(x)
print("JI done!")

BG done!
JI done!


In [17]:
for i in range(1,n-1):
    for j in range(i,n-1):
        print(i,j,len(JI[(i,j)]), len(BG[(i,j)]))



1 1 2 2
1 2 2 2
1 3 2 2
2 2 4 5
2 3 5 6
3 3 8 10


In [15]:
[x for x in bigrassmannians if x not in join_irreducibles]

[s3*s5*s4*s2*s3,
 s3*s5*s4*s2*s3*s1*s2,
 s3*s4*s2*s3*s5*s3*s4*s1*s2*s3,
 s2*s3*s5*s4*s1*s2*s3,
 s2*s3*s5*s4*s3*s1*s2]

In [29]:
def join_JI(S,i,j):
    S = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]   


    U = []
    for y in BG[(i,j)]:
        is_upper = True
        for s in S:
            if not s.bruhat_le(y):
                is_upper = False
        if is_upper == True:
            U.append(y)
    
    return (W_subposet(U)).minimal_elements()   

In [13]:
a = convert_from_123(345234123)
b = convert_from_123(34543234543)

join([a,b])

[s3*s4*s5*s2*s3*s4*s5*s3*s4*s1*s2*s3]

In [14]:
JM(s3*s4*s5*s2*s3*s4*s5*s3*s4*s1*s2*s3)

[s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3]

In [8]:
i = 0

JM_dict={}

for x in W:
    if x != e:
#        JM_dict[x] = JM(x)
        JM_x = X[x]

#        save(JM_x, 'JM_data/{}{}/JM_{}'.format(CartanType(W)[0],CartanType(W)[1],convert_to_123(x)))
#        with open('JM_data/{}{}.txt'.format(CartanType(W)[0],CartanType(W)[1]), "a") as myfile:
#            myfile.write('w={}\nJM(w)={} \n\n'.format(convert_to_123(x), [eval(convert_to_123(y)) for y in JM_dict[x]]   ))


        print("%d/%d"%(i,len(W)-1))
        i += 1
print("Done")

#save(JM_dict, "JM_data/%s" %(CartanType(W)[0]+str(CartanType(W)[1])))

print("Saved")

0/51839
1/51839
2/51839
3/51839
4/51839
5/51839
6/51839
7/51839
8/51839
9/51839
10/51839
11/51839
12/51839
13/51839
14/51839
15/51839
16/51839
17/51839
18/51839
19/51839
20/51839
21/51839
22/51839
23/51839
24/51839
25/51839
26/51839
27/51839
28/51839
29/51839
30/51839
31/51839
32/51839
33/51839
34/51839
35/51839
36/51839
37/51839
38/51839
39/51839
40/51839
41/51839
42/51839
43/51839
44/51839
45/51839
46/51839
47/51839
48/51839
49/51839
50/51839
51/51839
52/51839
53/51839
54/51839
55/51839
56/51839
57/51839
58/51839
59/51839
60/51839
61/51839
62/51839
63/51839
64/51839
65/51839
66/51839
67/51839
68/51839
69/51839
70/51839
71/51839
72/51839
73/51839
74/51839
75/51839
76/51839
77/51839
78/51839
79/51839
80/51839
81/51839
82/51839
83/51839
84/51839
85/51839
86/51839
87/51839
88/51839
89/51839
90/51839
91/51839
92/51839
93/51839
94/51839
95/51839
96/51839
97/51839
98/51839
99/51839
100/51839
101/51839
102/51839
103/51839
104/51839
105/51839
106/51839
107/51839
108/51839
109/51839
110/51839


832/51839
833/51839
834/51839
835/51839
836/51839
837/51839
838/51839
839/51839
840/51839
841/51839
842/51839
843/51839
844/51839
845/51839
846/51839
847/51839
848/51839
849/51839
850/51839
851/51839
852/51839
853/51839
854/51839
855/51839
856/51839
857/51839
858/51839
859/51839
860/51839
861/51839
862/51839
863/51839
864/51839
865/51839
866/51839
867/51839
868/51839
869/51839
870/51839
871/51839
872/51839
873/51839
874/51839
875/51839
876/51839
877/51839
878/51839
879/51839
880/51839
881/51839
882/51839
883/51839
884/51839
885/51839
886/51839
887/51839
888/51839
889/51839
890/51839
891/51839
892/51839
893/51839
894/51839
895/51839
896/51839
897/51839
898/51839
899/51839
900/51839
901/51839
902/51839
903/51839
904/51839
905/51839
906/51839
907/51839
908/51839
909/51839
910/51839
911/51839
912/51839
913/51839
914/51839
915/51839
916/51839
917/51839
918/51839
919/51839
920/51839
921/51839
922/51839
923/51839
924/51839
925/51839
926/51839
927/51839
928/51839
929/51839
930/51839
931/51839


1599/51839
1600/51839
1601/51839
1602/51839
1603/51839
1604/51839
1605/51839
1606/51839
1607/51839
1608/51839
1609/51839
1610/51839
1611/51839
1612/51839
1613/51839
1614/51839
1615/51839
1616/51839
1617/51839
1618/51839
1619/51839
1620/51839
1621/51839
1622/51839
1623/51839
1624/51839
1625/51839
1626/51839
1627/51839
1628/51839
1629/51839
1630/51839
1631/51839
1632/51839
1633/51839
1634/51839
1635/51839
1636/51839
1637/51839
1638/51839
1639/51839
1640/51839
1641/51839
1642/51839
1643/51839
1644/51839
1645/51839
1646/51839
1647/51839
1648/51839
1649/51839
1650/51839
1651/51839
1652/51839
1653/51839
1654/51839
1655/51839
1656/51839
1657/51839
1658/51839
1659/51839
1660/51839
1661/51839
1662/51839
1663/51839
1664/51839
1665/51839
1666/51839
1667/51839
1668/51839
1669/51839
1670/51839
1671/51839
1672/51839
1673/51839
1674/51839
1675/51839
1676/51839
1677/51839
1678/51839
1679/51839
1680/51839
1681/51839
1682/51839
1683/51839
1684/51839
1685/51839
1686/51839
1687/51839
1688/51839
1689/51839

2346/51839
2347/51839
2348/51839
2349/51839
2350/51839
2351/51839
2352/51839
2353/51839
2354/51839
2355/51839
2356/51839
2357/51839
2358/51839
2359/51839
2360/51839
2361/51839
2362/51839
2363/51839
2364/51839
2365/51839
2366/51839
2367/51839
2368/51839
2369/51839
2370/51839
2371/51839
2372/51839
2373/51839
2374/51839
2375/51839
2376/51839
2377/51839
2378/51839
2379/51839
2380/51839
2381/51839
2382/51839
2383/51839
2384/51839
2385/51839
2386/51839
2387/51839
2388/51839
2389/51839
2390/51839
2391/51839
2392/51839
2393/51839
2394/51839
2395/51839
2396/51839
2397/51839
2398/51839
2399/51839
2400/51839
2401/51839
2402/51839
2403/51839
2404/51839
2405/51839
2406/51839
2407/51839
2408/51839
2409/51839
2410/51839
2411/51839
2412/51839
2413/51839
2414/51839
2415/51839
2416/51839
2417/51839
2418/51839
2419/51839
2420/51839
2421/51839
2422/51839
2423/51839
2424/51839
2425/51839
2426/51839
2427/51839
2428/51839
2429/51839
2430/51839
2431/51839
2432/51839
2433/51839
2434/51839
2435/51839
2436/51839

3096/51839
3097/51839
3098/51839
3099/51839
3100/51839
3101/51839
3102/51839
3103/51839
3104/51839
3105/51839
3106/51839
3107/51839
3108/51839
3109/51839
3110/51839
3111/51839
3112/51839
3113/51839
3114/51839
3115/51839
3116/51839
3117/51839
3118/51839
3119/51839
3120/51839
3121/51839
3122/51839
3123/51839
3124/51839
3125/51839
3126/51839
3127/51839
3128/51839
3129/51839
3130/51839
3131/51839
3132/51839
3133/51839
3134/51839
3135/51839
3136/51839
3137/51839
3138/51839
3139/51839
3140/51839
3141/51839
3142/51839
3143/51839
3144/51839
3145/51839
3146/51839
3147/51839
3148/51839
3149/51839
3150/51839
3151/51839
3152/51839
3153/51839
3154/51839
3155/51839
3156/51839
3157/51839
3158/51839
3159/51839
3160/51839
3161/51839
3162/51839
3163/51839
3164/51839
3165/51839
3166/51839
3167/51839
3168/51839
3169/51839
3170/51839
3171/51839
3172/51839
3173/51839
3174/51839
3175/51839
3176/51839
3177/51839
3178/51839
3179/51839
3180/51839
3181/51839
3182/51839
3183/51839
3184/51839
3185/51839
3186/51839

...

48950/51839
48951/51839
48952/51839
48953/51839
48954/51839
48955/51839
48956/51839
48957/51839
48958/51839
48959/51839
48960/51839
48961/51839
48962/51839
48963/51839
48964/51839
48965/51839
48966/51839
48967/51839
48968/51839
48969/51839
48970/51839
48971/51839
48972/51839
48973/51839
48974/51839
48975/51839
48976/51839
48977/51839
48978/51839
48979/51839
48980/51839
48981/51839
48982/51839
48983/51839
48984/51839
48985/51839
48986/51839
48987/51839
48988/51839
48989/51839
48990/51839
48991/51839
48992/51839
48993/51839
48994/51839
48995/51839
48996/51839
48997/51839
48998/51839
48999/51839
49000/51839
49001/51839
49002/51839
49003/51839
49004/51839
49005/51839
49006/51839
49007/51839
49008/51839
49009/51839
49010/51839
49011/51839
49012/51839
49013/51839
49014/51839
49015/51839
49016/51839
49017/51839
49018/51839
49019/51839
49020/51839
49021/51839
49022/51839
49023/51839
49024/51839
49025/51839
49026/51839
49027/51839
49028/51839
49029/51839
49030/51839
49031/51839
49032/51839
4903

49633/51839
49634/51839
49635/51839
49636/51839
49637/51839
49638/51839
49639/51839
49640/51839
49641/51839
49642/51839
49643/51839
49644/51839
49645/51839
49646/51839
49647/51839
49648/51839
49649/51839
49650/51839
49651/51839
49652/51839
49653/51839
49654/51839
49655/51839
49656/51839
49657/51839
49658/51839
49659/51839
49660/51839
49661/51839
49662/51839
49663/51839
49664/51839
49665/51839
49666/51839
49667/51839
49668/51839
49669/51839
49670/51839
49671/51839
49672/51839
49673/51839
49674/51839
49675/51839
49676/51839
49677/51839
49678/51839
49679/51839
49680/51839
49681/51839
49682/51839
49683/51839
49684/51839
49685/51839
49686/51839
49687/51839
49688/51839
49689/51839
49690/51839
49691/51839
49692/51839
49693/51839
49694/51839
49695/51839
49696/51839
49697/51839
49698/51839
49699/51839
49700/51839
49701/51839
49702/51839
49703/51839
49704/51839
49705/51839
49706/51839
49707/51839
49708/51839
49709/51839
49710/51839
49711/51839
49712/51839
49713/51839
49714/51839
49715/51839
4971

50316/51839
50317/51839
50318/51839
50319/51839
50320/51839
50321/51839
50322/51839
50323/51839
50324/51839
50325/51839
50326/51839
50327/51839
50328/51839
50329/51839
50330/51839
50331/51839
50332/51839
50333/51839
50334/51839
50335/51839
50336/51839
50337/51839
50338/51839
50339/51839
50340/51839
50341/51839
50342/51839
50343/51839
50344/51839
50345/51839
50346/51839
50347/51839
50348/51839
50349/51839
50350/51839
50351/51839
50352/51839
50353/51839
50354/51839
50355/51839
50356/51839
50357/51839
50358/51839
50359/51839
50360/51839
50361/51839
50362/51839
50363/51839
50364/51839
50365/51839
50366/51839
50367/51839
50368/51839
50369/51839
50370/51839
50371/51839
50372/51839
50373/51839
50374/51839
50375/51839
50376/51839
50377/51839
50378/51839
50379/51839
50380/51839
50381/51839
50382/51839
50383/51839
50384/51839
50385/51839
50386/51839
50387/51839
50388/51839
50389/51839
50390/51839
50391/51839
50392/51839
50393/51839
50394/51839
50395/51839
50396/51839
50397/51839
50398/51839
5039

51005/51839
51006/51839
51007/51839
51008/51839
51009/51839
51010/51839
51011/51839
51012/51839
51013/51839
51014/51839
51015/51839
51016/51839
51017/51839
51018/51839
51019/51839
51020/51839
51021/51839
51022/51839
51023/51839
51024/51839
51025/51839
51026/51839
51027/51839
51028/51839
51029/51839
51030/51839
51031/51839
51032/51839
51033/51839
51034/51839
51035/51839
51036/51839
51037/51839
51038/51839
51039/51839
51040/51839
51041/51839
51042/51839
51043/51839
51044/51839
51045/51839
51046/51839
51047/51839
51048/51839
51049/51839
51050/51839
51051/51839
51052/51839
51053/51839
51054/51839
51055/51839
51056/51839
51057/51839
51058/51839
51059/51839
51060/51839
51061/51839
51062/51839
51063/51839
51064/51839
51065/51839
51066/51839
51067/51839
51068/51839
51069/51839
51070/51839
51071/51839
51072/51839
51073/51839
51074/51839
51075/51839
51076/51839
51077/51839
51078/51839
51079/51839
51080/51839
51081/51839
51082/51839
51083/51839
51084/51839
51085/51839
51086/51839
51087/51839
5108

51688/51839
51689/51839
51690/51839
51691/51839
51692/51839
51693/51839
51694/51839
51695/51839
51696/51839
51697/51839
51698/51839
51699/51839
51700/51839
51701/51839
51702/51839
51703/51839
51704/51839
51705/51839
51706/51839
51707/51839
51708/51839
51709/51839
51710/51839
51711/51839
51712/51839
51713/51839
51714/51839
51715/51839
51716/51839
51717/51839
51718/51839
51719/51839
51720/51839
51721/51839
51722/51839
51723/51839
51724/51839
51725/51839
51726/51839
51727/51839
51728/51839
51729/51839
51730/51839
51731/51839
51732/51839
51733/51839
51734/51839
51735/51839
51736/51839
51737/51839
51738/51839
51739/51839
51740/51839
51741/51839
51742/51839
51743/51839
51744/51839
51745/51839
51746/51839
51747/51839
51748/51839
51749/51839
51750/51839
51751/51839
51752/51839
51753/51839
51754/51839
51755/51839
51756/51839
51757/51839
51758/51839
51759/51839
51760/51839
51761/51839
51762/51839
51763/51839
51764/51839
51765/51839
51766/51839
51767/51839
51768/51839
51769/51839
51770/51839
5177

In [16]:
load("JM_data/{}/JM_{}".format(CartanType(W)[0]+str(CartanType(W)[1]), convert_to_123(w0)))

[s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6,
 s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1,
 s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3,
 s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5,
 s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4,
 s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2]